In [2]:
import pandas as pd
import numpy as np

### 0. Feature Selection
Deciding which features are truly important for the initial clustering and which will either complement the models or be used to evaluate their own density in the final clusters. 

In [12]:
customer_info = pd.read_csv('../data/raw/clean_customer_info.csv', index_col="customer_id")

In [13]:
customer_info.head()

,customer_gender,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,lifetime_spend_vegetables,lifetime_spend_nonalcohol_drinks,lifetime_spend_alcohol_drinks,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,age
customer_id,,,,,,,,,,,,,,,,,,,
29930,-1.0,1.0,1.0,0.0,0.5,-0.603317,0.350843,0.000000,0.299376,0.717154,0.411301,0.716551,0.629503,-0.301504,0.254423,0.467872,0.867906,0.000000,-0.032258
6813,0.0,-1.0,0.0,-1.0,0.5,-0.514208,-2.423571,0.142857,0.354116,-1.414715,-1.445960,-0.272893,-0.466921,-0.438051,-1.433625,-0.486431,-0.384736,2.460565,0.838710
39451,-1.0,1.0,2.0,-1.0,2.0,-0.014666,0.202372,0.142857,0.790758,0.734816,0.740321,0.473346,0.655227,0.343099,0.048656,-0.729704,0.723964,0.000000,-0.612903
21557,0.0,-1.0,-1.0,0.0,-1.0,-0.141192,0.940175,0.142857,-1.730063,-0.495624,0.733403,0.031924,0.061145,-1.395136,0.751694,-0.220805,0.447107,0.048761,-0.193548
16415,0.0,0.0,0.0,0.0,1.0,0.959008,0.042696,0.142857,-0.350786,0.449038,-0.192228,0.090877,0.475513,0.461414,-1.038064,-1.158433,0.521750,0.192977,0.129032


### 1. Initial Individual Clustering (Baseline Models)

### 2. Tandem Approaches (Two-Step Clustering)

#### 2.1) Hierarchical + K-Means

#### 2.2) K-Means (large k) + Hierarchical

### 3. Comparison, Profiling and Selection